In [ ]:
%load_ext autoreload
%autoreload 2

## Imports

In [ ]:
from dmsbatch import create_batch_client, create_blob_client

## First create a batch client from the config file

In [ ]:
client = create_batch_client('../tests/data/dmsbatch.config')
blob_client = create_blob_client('../tests/data/dmsbatch.config')

## Application packages
To copy large files and programs it is best to zip (or targz) them and upload them as application packages

Application packages are setup separately in either azure management apis or from the web console or cli tool

These are referenced here by their name and version
e.g. DSM2, python and other programs

In [ ]:
app_pkgs = [('dsm2linux', '8.2.8449db2', 'DSM2-8.2.8449db2-Linux/bin')]

### Create or resize existing pool
If the pool doesn't exist it will create it
If the pool exists, it will resize to the second arg

In [ ]:
pool_start_cmds = ['printenv',
'yum install -y glibc.i686 libstdc++.i686 glibc.x86_64 libstdc++.x86_64',# --setopt=protected_multilib=false',
'yum-config-manager --add-repo https://yum.repos.intel.com/2019/setup/intel-psxe-runtime-2019.repo',
'rpm --import https://yum.repos.intel.com/2019/setup/RPM-GPG-KEY-intel-psxe-runtime-2019',
'yum install -y intel-icc-runtime-32bit intel-ifort-runtime-32bit']
client.wrap_commands_in_shell('linux',pool_start_cmds)

In [ ]:
created = client.create_pool('dsm2linuxpool',
                             1,
                             app_packages=[(app, version)
                                           for app, version, _ in app_pkgs],
                             vm_size='standard_d2d_v4',  # vm_size='standard_f2s_v2',
                             tasks_per_vm=2,
                             os_image_data=('openlogic', 'centos', '7_8'),
                             start_task_cmd=client.wrap_commands_in_shell(
                                 pool_start_cmds, ostype='linux'),
                             start_task_admin=True,
                             elevation_level='admin'
                             )

In [ ]:
if not created:
    client.resize_pool('dsm2linuxpool',1)

In [ ]:
client.wait_for_pool_nodes('dsm2linuxpool')

### Create job on pool or fail if it exists
Jobs are containers of tasks (things that run on nodes (machines) in the pool). If this exists, the next line will fail

In [ ]:
try:
    client.create_job('dsm2linuxjobs','dsm2linuxpool')
except Exception as exc:
    print('Job already exists? Will continue anyways')

### Upload input files
Zip the input files and add them to storage container (in this case the auto storage associated with the batch account)

In [ ]:
import os, datetime
userid = os.getlogin()
tsnow = str(datetime.datetime.now().timestamp()).split('.')[0]
task_name = f'hydro_run_{tsnow}_{userid}'
print(task_name)

In [ ]:
local_dir = '../tests/data/dsm2v821'
input_file=blob_client.zip_and_upload('dsm2linuxjobs',f'{task_name}/dsm2v821.zip',local_dir,30)

In [ ]:
input_file = client.create_input_file_spec('dsm2linuxjobs',blob_prefix=f'{task_name}/dsm2v821.zip',file_path='.')

In [ ]:
import dmsbatch
permissions = dmsbatch.commands.BlobSasPermissions(write=True)
# |helpers.azureblob.BlobPermissions.ADD|helpers.azureblob.BlobPermissions.CREATE
output_dir_sas_url = blob_client.get_container_sas_url('dsm2linuxjobs', permissions)
print(output_dir_sas_url)

In [ ]:
std_out_files = client.create_output_file_spec('../std*.txt', output_dir_sas_url, blob_path=f'{task_name}')
output_dir = client.create_output_file_spec('**/output/*',output_dir_sas_url, blob_path=f'{task_name}')

### Create a task
This uses the application package as pre -set up. If not, create one https://docs.microsoft.com/en-us/azure/batch/batch-application-packages

In [ ]:
# tsnow = str(datetime.datetime.now().timestamp()).split('.')[0]
# task_name = f'hydro_run_{tsnow}'
set_path_string = client.set_path_to_apps(app_pkgs, ostype='linux')
cmd_string = client.wrap_cmd_with_app_path(f"""source /opt/intel/psxe_runtime/linux/bin/compilervars.sh ia32;
{set_path_string};
cd {task_name};
unzip dsm2v821.zip;
rm dsm2v821.zip;
cd study_templates/historical;
hydro hydro.inp;
qual qual_ec.inp;
    """, app_pkgs, ostype='linux')
hydro_task=client.create_task(task_name, cmd_string, 
    resource_files=[input_file], output_files=[std_out_files, output_dir])


In [ ]:
task_name

### Next submit the task and wait 

In [ ]:
client.submit_tasks('dsm2linuxjobs',[hydro_task])

In [ ]:
client.wait_for_tasks_to_complete('dsm2linuxjobs',timeout=datetime.timedelta(minutes=80),polling_interval_secs=60)

## Finally resize the pool to 0 to save costs

In [ ]:
client.resize_pool('dsm2linuxpool',0)